# Hurricane Ike Maximum Water Levels
Compute the maximum water level during Hurricane Ike on a 9 million node triangular mesh storm surge model.  Plot the results using [HoloViz](https://holoviz.org/) TriMesh rendering with Datashader. 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import fsspec

### Start a dask cluster to crunch the data

In [ ]:
from dask.distributed import Client, progress

In [ ]:
import sys, os
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

worker_max=20
client,cluster=ebd.start_dask_cluster(profile='esip-qhub',worker_max=worker_max,
                                      worker_profile='Pangeo Worker',
                                      propagate_env=True,adaptive_scaling=False)  

In [ ]:
cluster

For demos, I often click in this cell and do "Cell=>Run All Above", then wait until the workers appear.  This can take several minutes (up to 6!) for instances to spin up and Docker containers to be downloaded. Then I shutdown the notebook and run again from the beginning, and the workers will fire up quickly because the instances have not spun down yet. 

In [ ]:
#cluster.adapt(minimum=4, maximum=20);

In [ ]:
#client = Client(cluster)

### Read the data using the cloud-friendly zarr data format

In [ ]:
ds = xr.open_zarr(fsspec.get_mapper('s3://pangeo-data-uswest2/esip/adcirc/ike', anon=False, requester_pays=True))

In [ ]:
#ds = xr.open_zarr(fsspec.get_mapper('gcs://pangeo-data/rsignell/adcirc_test01'))

In [ ]:
ds['zeta']

How many GB of sea surface height data do we have?

In [ ]:
ds['zeta'].nbytes/1.e9

Take the maximum over the time dimension and persist the data on the workers to use later.  This is the computationally intensive step.

In [ ]:
%%time
max_var = ds['zeta'].max(dim='time').persist()

### Visualize data on mesh using HoloViz.org tools

In [ ]:
import numpy as np
import datashader as dshade
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import hvplot.xarray
import holoviews.operation.datashader as dshade

dshade.datashade.precompute = True
hv.extension('bokeh')

In [ ]:
v = np.vstack((ds['x'], ds['y'], max_var)).T
verts = pd.DataFrame(v, columns=['x','y','vmax'])

In [ ]:
points = gv.operation.project_points(gv.Points(verts, vdims=['vmax']))

In [ ]:
tris = pd.DataFrame(ds['element'].values.astype('int')-1, columns=['v0','v1','v2'])

In [ ]:
tiles = gv.tile_sources.OSM

In [ ]:
value = 'max water level'
label = '{} (m)'.format(value)
trimesh = gv.TriMesh((tris, points), label=label)
mesh = dshade.rasterize(trimesh).opts(
              cmap='rainbow', colorbar=True, width=600, height=400)

In [ ]:
tiles * mesh

### Extract a time series at a specified lon, lat location

Because Xarray does not yet understand that `x` and `y` are coordinate variables on this triangular mesh, we create our own simple function to find the closest point. If we had a lot of these, we could use a more fancy tree algorithm.

In [ ]:
# find the indices of the points in (x,y) closest to the points in (xi,yi)
def nearxy(x,y,xi,yi):
    ind = np.ones(len(xi),dtype=int)
    for i in range(len(xi)):
        dist = np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i] = dist.argmin()
    return ind

In [ ]:
#just offshore of Galveston
lat = 29.2329856
lon = -95.1535041

In [ ]:
ind = nearxy(ds['x'].values,ds['y'].values,[lon], [lat])

In [ ]:
ds['zeta'][:,ind].hvplot(x='time', grid=True)